In [ ]:
#hide   
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
from pprint import pprint as pp
import rfpy
from rfpy.parser import *
from rfpy.utils import *
from rfpy.blocks import *
from nbdev.showdoc import *
from fastcore.xtras import Path
from fastcore.foundation import L
import numpy as np
import pandas as pd

In [ ]:
#hide
%load_ext autoreload
%autoreload 2 

In [ ]:
entrada = Path(r'C:\Users\rsilva\Downloads\entrada')
saida = Path(r'C:\Users\rsilva\Downloads\saida')

In [ ]:
arquivos = get_files(entrada) ;  arquivos

In [ ]:
one_tid = arquivos[0]
mult_tid = arquivos[1]

On a normal conda environment the time of the following cell was
```python
%%timeit
one = process_bin(one_tid)
```
1.22 s ± 17.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
%%timeit
one = parse_bin(one_tid)

On a normal conda environment the time of the following cell was
```python
%%timeit
multi = process_bin(mult_tid)
```
1.29 s ± 73.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

In [ ]:
%%timeit
multi = parse_bin(mult_tid)

In [ ]:
one = parse_bin(one_tid)

In [ ]:
one.keys()

In [ ]:
one['blocos']

In [ ]:
multi = parse_bin(mult_tid)

In [ ]:
multi.keys()

In [ ]:
multi['file_version']

In [ ]:
multi['string']

In [ ]:
multi['blocos']

In [ ]:
x = multi['blocos'][(63,40)][0]

In [ ]:
getattrs(x)

{'agc_array': '30-34-39-48-48-46-42',
 'bw': 40,
 'data_points': 2048,
 'datetime_stamp': datetime.datetime(2020, 12, 1, 15, 45, 9, 816463),
 'global_error_code': b'ff',
 'global_flags_code': 255,
 'group_id': 0,
 'id_antenna': 0,
 'level_offset': -20,
 'n_agc': 7,
 'n_padding': -20,
 'n_tunning': 7,
 'num_meas': 1,
 'passo': 0.019540791402051783,
 'processing': 'average',
 'rbw': 19540,
 'size': 2136,
 'spent_time_microsecs': 5570,
 'start_channel': 0,
 'start_mega': 70,
 'start_mili': 0,
 'stop_channel': 0,
 'stop_mega': 110,
 'stop_mili': 0,
 'thread_id': 40,
 'tunning_info': 'completed without error',
 'type': 63,
 'unit': '%'}

In [ ]:
%%time
b1 = multi['blocos'][(63,40)].map(getattrs)

In [ ]:
df = pd.DataFrame(b1).set_index('datetime_stamp') ; df.head()

In [ ]:
b2 = multi['blocos'][(63,50)].map(getattrs)

In [ ]:
df2 = pd.DataFrame(b2).set_index('datetime_stamp') ; df2.head()

In [ ]:
for col in df.columns:
    print(col, df[col].unique())